<a href="https://colab.research.google.com/github/ThegreatShible/Bagging-of-Density-Estimators/blob/master/Test_Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generation Sample

Generate a Pandas DataFrame containing three columns : strategy id, date, and p&l

- the strategy_id should be incremental integers
- the data should represent 1000 strategies with daily records from january 1,2000 to today
- the p&l daily values should be randomly assigned as ether -1 or +1


In [22]:
import pandas as pd
import numpy as np

# Définir les dates de début et de fin
start_date = '2000-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

# Générer la plage de dates
dates = pd.date_range(start=start_date, end=end_date)

# Nombre de stratégies
num_strategies = 1000

# Créer une liste pour les strategy_id
strategy_ids = np.arange(1, num_strategies + 1)

# Générer le DataFrame
data = {
    'strategy_id': np.repeat(strategy_ids, len(dates)),  # Répète chaque strategy_id pour chaque date
    'date': np.tile(dates, num_strategies),  # Répète la série de dates pour chaque strategy_id
    'p_l': np.random.choice([-1, 1], size=len(dates) * num_strategies)  # Génère des valeurs aléatoires de -1 ou +1
}

df = pd.DataFrame(data)

# Afficher les premières lignes du DataFrame
print(df.head())
# Afficher les dernières lignes du DataFrame
print(df.tail())
# Sauvegarde du df dans un fichier CSV
df.to_csv('/content/df.csv', index=False)

   strategy_id       date  p_l
0            1 2000-01-01    1
1            1 2000-01-02    1
2            1 2000-01-03   -1
3            1 2000-01-04    1
4            1 2000-01-05    1
         strategy_id       date  p_l
8959995         1000 2024-07-08    1
8959996         1000 2024-07-09    1
8959997         1000 2024-07-10    1
8959998         1000 2024-07-11    1
8959999         1000 2024-07-12   -1


# MTD/YTD Capital Gain

Month to date (MTD) and Year to Date (YTD) refer to the periods of time starting from the first day of the current motnh or year, respectively and ending at the current date. they include all days from the beginning of the period up to and including the current day. The p&l should be summed over the period to obtain the aggregated p&l. Write 2 functions (DuckDB and Pandas) to evaluate the MTD and YTD for each date.



## Pandas

In [20]:
import pandas as pd

def calculate_mtd_ytd_pandas(data: pd.DataFrame):
    data['date'] = pd.to_datetime(data['date'])

    # Calcul du MTD
    data['month'] = data['date'].dt.to_period('M')
    data['mtd'] = data.groupby(['strategy_id', 'month'])['p_l'].cumsum()

    # Calcul du YTD
    data['year'] = data['date'].dt.to_period('Y')
    data['ytd'] = data.groupby(['strategy_id', 'year'])['p_l'].cumsum()

    return data[['strategy_id', 'date', 'p_l', 'mtd', 'ytd']]

# Tester la fonction Pandas
result_pandas = calculate_mtd_ytd_pandas(df)
print(result_pandas.head())


   strategy_id       date  p_l  mtd  ytd
0            1 2000-01-01    1    1    1
1            1 2000-01-02   -1    0    0
2            1 2000-01-03    1    1    1
3            1 2000-01-04   -1    0    0
4            1 2000-01-05   -1   -1   -1


## DuckDB

Simulation de la création d'une table DuckDB à partir du df

In [23]:
import duckdb

# Chemin vers le fichier CSV dans Google Colab
csv_file = '/content/df.csv'

# Connexion à DuckDB
conn = duckdb.connect(database=':memory:', read_only=False)

# Création d'une table DuckDB à partir du fichier CSV
create_table_query = f"""
CREATE TABLE transactions AS FROM READ_CSV('df.csv');
"""
conn.execute(create_table_query)

# Vérifier que la table a été créée
tables = conn.execute("SHOW TABLES").fetch_df()
print(tables)

# Affichage du contenu de la table
query = "SELECT * FROM transactions"
result = conn.execute(query).fetch_df()
print(result)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

           name
0  transactions
         strategy_id       date  p_l
0                  1 2000-01-01    1
1                  1 2000-01-02    1
2                  1 2000-01-03   -1
3                  1 2000-01-04    1
4                  1 2000-01-05    1
...              ...        ...  ...
8959995         1000 2024-07-08    1
8959996         1000 2024-07-09    1
8959997         1000 2024-07-10    1
8959998         1000 2024-07-11    1
8959999         1000 2024-07-12   -1

[8960000 rows x 3 columns]


Création de la requête afin de calculer MTD et YTD

In [5]:
# Requête pour calculer MTD et YTD
query = """
WITH mtd_calc AS (
    SELECT
        strategy_id,
        date,
        p_l,
        SUM(p_l) OVER (PARTITION BY strategy_id, DATE_TRUNC('month', date) ORDER BY date) AS mtd
    FROM transactions
),
ytd_calc AS (
    SELECT
        strategy_id,
        date,
        p_l,
        SUM(p_l) OVER (PARTITION BY strategy_id, DATE_TRUNC('year', date) ORDER BY date) AS ytd
    FROM transactions
)
SELECT
    t.strategy_id,
    t.date,
    t.p_l,
    m.mtd,
    y.ytd
FROM
    transactions t
JOIN
    mtd_calc m ON t.strategy_id = m.strategy_id AND t.date = m.date
JOIN
    ytd_calc y ON t.strategy_id = y.strategy_id AND t.date = y.date
ORDER BY
    t.strategy_id, t.date
"""

# Exécution de la requête et récupération des résultats
result = conn.execute(query).fetch_df()

# Afficher les résultats
print(result)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

         strategy_id       date  p_l  mtd  ytd
0                  1 2000-01-01   -1 -1.0 -1.0
1                  1 2000-01-02   -1 -2.0 -2.0
2                  1 2000-01-03   -1 -3.0 -3.0
3                  1 2000-01-04    1 -2.0 -2.0
4                  1 2000-01-05   -1 -3.0 -3.0
...              ...        ...  ...  ...  ...
8959995         1000 2024-07-08   -1 -2.0 -2.0
8959996         1000 2024-07-09   -1 -3.0 -3.0
8959997         1000 2024-07-10   -1 -4.0 -4.0
8959998         1000 2024-07-11   -1 -5.0 -5.0
8959999         1000 2024-07-12   -1 -6.0 -6.0

[8960000 rows x 5 columns]


# Large-scale data


Discuss strategies for maning the same calculation with a dataset that is too large to fit into memory ?



Pour répondre à cette question, je pense qu’il faudrait d’abord comprendre ce qu’on entend par « trop volumineux pour tenir en mémoire ».

"trop volumineuses pour tenir en mémoire", cela signifie que la quantité de données dépasse la capacité de la mémoire vive (RAM) disponible sur un seul système informatique. La mémoire vive est l'espace de stockage temporaire utilisé par le système d'exploitation pour stocker des données nécessaires au traitement.

Face à cette contrainte, plusieurs stratégies peuvent être adoptées :
-	**Partitionnement** : Diviser les données en morceaux gérables (partitions) qui peuvent être traités séparément. Exemples : BigQuery, PostgreSQL avec partitionnement
- **Traitement distribué**: Utiliser des systèmes distribués comme Hadoop, Spark ou Cassandra pour répartir les données sur plusieurs nœuds (ie des serveurs physiques ou virtuels)
- **Batch Processing** : Traiter les données par lots, en lisant et en écrivant les données de manière séquentielle plutôt que simultanément. Exemple : Apache Beam.
- **Calcul sur disque** : Utiliser des outils conçus pour manipuler efficacement les données stockées sur disque plutôt que celles en mémoire vive. Par exemple, le Chunking permet de lire et de traiter les données en morceaux (chunks) plutôt qu'en une seule fois. Des bibliothèques comme Dask en Python permettent également le calcul parallèle et distribué, adapté à des DataFrames plus grands que la mémoire disponible.
